In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import statistics
from datetime import datetime
from pandas import Series
import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

import seaborn as sns
import os
import statistics

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import TimeSeriesSplit

from dateutil.relativedelta import relativedelta 
from scipy.optimize import minimize              

import statsmodels.formula.api as smf          
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

from itertools import product      

from statsmodels.tsa.api import ExponentialSmoothing,SimpleExpSmoothing, Holt

from statsmodels.tsa.stattools import adfuller
from matplotlib.pylab import rcParams

from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
import statsmodels.api as sm

In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from pandas.plotting import register_matplotlib_converters
from sklearn.preprocessing import RobustScaler

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [3]:
def create_dataset(X, y, time_steps = 1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [4]:
from math import sqrt

def rmse(y_true, y_pred):
    squared_error = 0
    for i in range(len(y_true)):
        squared_error = squared_error + (y_true[i] - y_pred[i]) ** 2
    root_mean_squared_error = sqrt(squared_error / len(y_true))
    return root_mean_squared_error

In [5]:
def moving_avg(rounds):
    y_hat['avg'] = 0
    for i in range(len(Valid)):
        y_pred = 0
        for j in range(rounds):
            y_pred = y_pred + Train.footfall[len(Train) - (j + 1) * len(Valid) + i]
        y_hat['avg'][i] = y_pred / rounds

In [6]:
def test_stationary(timeseries):
    #Determine rolling statistics
    rolmean = pd.Series(timeseries).rolling(window = 28).mean()
    rolstd = pd.Series(timeseries).rolling(window = 28).std()
    
    #Plot rolling Statistics
    orig = plt.plot(timeseries, color = "blue", label = "Original")
    mean = plt.plot(rolmean, color = "red", label = "Rolling Mean")
    std = plt.plot(rolstd, color = "black", label = "Rolling Std")
    plt.legend(loc = "best")
    plt.title("Rolling Mean and Standard Deviation")
    plt.show(block = False)
    
    #Perform Dickey Fuller test
    print("Results of Dickey Fuller test: ")
    dftest = adfuller(timeseries, autolag = 'AIC')
    dfoutput = pd.Series(dftest[0:4], index = ['Test Statistics', 'p-value', '# Lag Used', 'Number of Observations Used'])
    
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)' %key] = value
    print(dfoutput)

In [7]:
def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "MAPE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    
    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    # Length of lists (as real numbers)
    T = float(len(actual_lst))
    
    # construct d according to crit
    if (crit == "MSE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(((actual - p1))**(power))
            e2_lst.append(((actual - p2))**(power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt

### Exclude Locations with Too Much Missing Data:

In [14]:
ff = pd.read_csv('stackfootfall.csv')
ff.timestamp = pd.to_datetime(ff.timestamp, format = '%Y-%m-%d %H:%M:%S') 

In [15]:
ff = ff.loc[(ff.timestamp >= '2018-01') & (ff.timestamp <= '2019-08-18 23:00')]

ff_count = ff.groupby('location')['footfall'].count() / max(ff.groupby('location')['footfall'].count())
limit1 = ff_count.loc[ff_count >= 0.6].to_frame()
ff_test = ff.loc[(ff.timestamp >= '2019-07-29') & (ff.timestamp <= '2019-08-18 23:00')]
ff_test_count = ff_test.groupby('location')['footfall'].count() / max(ff_test.groupby('location')['footfall'].count())
limit2 = ff_test_count.loc[ff_test_count >= 0.8].to_frame()
limit = pd.merge(limit1, limit2, how = 'inner', on = 'location')

ff = pd.merge(ff, limit, how = 'inner', on = 'location')
ff

,Unnamed: 0,timestamp,location,device,footfall,year,month,day,hour,day_of_week,footfall_x,footfall_y
0,1344,2018-01-01 00:00:00,5.0,1780.0,494.0,2018,1,1,0,1,0.892256,1.0
1,1345,2018-01-01 01:00:00,5.0,1780.0,899.0,2018,1,1,1,1,0.892256,1.0
2,1346,2018-01-01 02:00:00,5.0,1780.0,770.0,2018,1,1,2,1,0.892256,1.0
3,1347,2018-01-01 03:00:00,5.0,1780.0,599.0,2018,1,1,3,1,0.892256,1.0
4,1348,2018-01-01 04:00:00,5.0,1780.0,331.0,2018,1,1,4,1,0.892256,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3818059,233635,2019-08-18 19:00:00,1168.0,827.0,42.0,2019,8,18,19,7,0.619529,1.0
3818060,233636,2019-08-18 20:00:00,1168.0,827.0,10.0,2019,8,18,20,7,0.619529,1.0
3818061,233637,2019-08-18 21:00:00,1168.0,827.0,13.0,2019,8,18,21,7,0.619529,1.0
3818062,233638,2019-08-18 22:00:00,1168.0,827.0,1.0,2019,8,18,22,7,0.619529,1.0


In [10]:
pred_accuracy = pd.DataFrame(columns = ('location', 'Naive', 'Moving_Average', 'Expoential_Smoothing', 
                                        'SARIMA', 'LSTM'))

dmtest = pd.DataFrame(columns = ('location', 'nm', 'p_nm', 'ne', 'p_ne', 'ns', 'p_ns', 
                                 'nl', 'p_nl', 'me', 'p_me', 'ms', 'p_ms', 'ml', 'p_ml', 
                                 'es', 'p_es', 'el', 'p_el', 'sl', 'p_sl'))

In [16]:
ff.location.nunique()

304

In [17]:
ff.location.unique()

array([   5.,    6.,    7.,   31.,   54.,   62.,   78.,   87.,   88.,
         89.,   91.,   92.,   97.,  108.,  111.,  112.,  114.,  115.,
        116.,  117.,  119.,  123.,  126.,  127.,  145.,  146.,  147.,
        152.,  155.,  165.,  185.,  186.,  222.,  233.,  237.,  266.,
        268.,  273.,  281.,  305.,  306.,  357.,  376.,  389.,  390.,
        409.,  414.,  419.,  421.,  425.,  429.,  445.,  447.,  448.,
        450.,  451.,  488.,  489.,  490.,  491.,  493.,  495.,  497.,
        498.,  500.,  501.,  502.,  503.,  504.,  509.,  512.,  515.,
        516.,  519.,  521.,  522.,  523.,  524.,  526.,  527.,  536.,
        541.,  542.,  544.,  546.,  547.,  551.,  554.,  556.,  560.,
        562.,  563.,  565.,  566.,  573.,  574.,  576.,  579.,  581.,
        584.,  586.,  587.,  588.,  589.,  592.,  593.,  597.,  598.,
        599.,  605.,  608.,  610.,  611.,  614.,  618.,  620.,  634.,
        639.,  642.,  649.,  657.,  660.,  705.,  710.,  711.,  712.,
        714.,  715.,

In [7]:
ff.location.unique()

array([   5.,    6.,    7.,   31.,   54.,   62.,   78.,   87.,   88.,
         89.,   91.,   92.,   97.,  108.,  111.,  112.,  114.,  115.,
        116.,  117.,  119.,  123.,  126.,  127.,  145.,  146.,  147.,
        152.,  155.,  165.,  185.,  186.,  222.,  233.,  237.,  266.,
        268.,  273.,  281.,  305.,  306.,  357.,  376.,  389.,  390.,
        409.,  414.,  419.,  421.,  425.,  429.,  445.,  447.,  448.,
        450.,  451.,  488.,  489.,  490.,  491.,  493.,  495.,  497.,
        498.,  500.,  501.,  502.,  503.,  504.,  509.,  512.,  515.,
        516.,  519.,  521.,  522.,  523.,  524.,  526.,  527.,  536.,
        541.,  542.,  544.,  546.,  547.,  551.,  554.,  556.,  560.,
        562.,  563.,  565.,  566.,  573.,  574.,  576.,  579.,  581.,
        584.,  586.,  587.,  588.,  589.,  592.,  593.,  597.,  598.,
        599.,  605.,  608.,  610.,  611.,  614.,  618.,  620.,  634.,
        639.,  642.,  649.,  657.,  660.,  705.,  710.,  711.,  712.,
        714.,  715.,

In [12]:
for loc in ff.location.unique()[326:]:

    ff_loc = ff.loc[(ff.location <= loc) & (ff.location >= loc)][[
        'timestamp','footfall','year','month','day','hour','day_of_week']]    
    ff_loc.index = ff_loc.timestamp
    ff_loc = ff_loc.resample('H').mean()

    ff_loc['footfall'].replace(0, np.nan, inplace = True)
    ff_loc['footfall'] = ff_loc['footfall'].interpolate(method='linear')
    ff_loc = ff_loc.reset_index(level = ['timestamp'])
    ff_loc.timestamp = pd.to_datetime(ff_loc.timestamp, format = '%Y-%m-%d %H:%M:%S')

    temp = ff_loc[['timestamp', 'footfall']]
    temp.index = temp.timestamp
    temp['year'] = temp.timestamp.dt.year
    temp['month'] = temp.timestamp.dt.month
    temp['day'] = temp.timestamp.dt.day
    temp['hour'] = temp.timestamp.dt.hour
    temp['day_of_week'] = temp.timestamp.dt.dayofweek + 1

    Train = temp.loc[(temp.timestamp >= '2018-01-01 00:00') & (temp.timestamp <= '2019-07-28 23:00')]
    Valid = temp.loc[(temp.timestamp >= '2019-07-29 00:00') & (temp.timestamp <= '2019-08-18 23:00')]
    
    if len(Valid) > 0:
        train = Train.resample('D').mean()
        valid = Valid.resample('D').mean() 

        train['footfall'] = train['footfall'].interpolate(method='linear')
        valid['footfall'] = valid['footfall'].interpolate(method='linear')

        # Monday to Thursday Ratio
        mtt = Train.loc[(Train.day_of_week >= 1) & (Train.day_of_week <= 4)].groupby('hour')['footfall'].mean().to_frame()
        mtt['ratio'] = mtt['footfall']/mtt['footfall'].sum()
        mtt = mtt.reset_index(level = ['hour'])

        # Friday Ratio
        fri = Train.loc[(Train.day_of_week >= 5) & (Train.day_of_week <= 5)].groupby('hour')['footfall'].mean().to_frame()
        fri['ratio'] = fri['footfall']/fri['footfall'].sum()
        fri = fri.reset_index(level = ['hour'])

        # Sat Ratio
        sat = Train.loc[(Train.day_of_week >= 6) & (Train.day_of_week <= 6)].groupby('hour')['footfall'].mean().to_frame()
        sat['ratio'] = sat['footfall']/sat['footfall'].sum()
        sat = sat.reset_index(level = ['hour'])

        # Sun Ratio
        sun = Train.loc[(Train.day_of_week >= 7) & (Train.day_of_week <= 7)].groupby('hour')['footfall'].mean().to_frame()
        sun['ratio'] = sun['footfall']/sun['footfall'].sum()
        sun = sun.reset_index(level = ['hour'])

        ratio1 = mtt[['hour','ratio']]
        ratio1['day_of_week'] = 1
        ratio2 = mtt[['hour','ratio']]
        ratio2['day_of_week'] = 2
        ratio3 = mtt[['hour','ratio']]
        ratio3['day_of_week'] = 3
        ratio4 = mtt[['hour','ratio']]
        ratio4['day_of_week'] = 4
        ratio5 = fri[['hour','ratio']] 
        ratio5['day_of_week'] = 5
        ratio6 = sat[['hour','ratio']]
        ratio6['day_of_week'] = 6
        ratio7 = sun[['hour','ratio']]
        ratio7['day_of_week'] = 7

        ratio_week = ratio1.append(ratio2).append(ratio3).append(ratio4).append(ratio5).append(ratio6).append(ratio7)

        merge = pd.merge(Valid, ratio_week, on = ('hour','day_of_week'), how = 'left')

        # Naive Approach
        dd = np.asarray(Train['footfall'])
        y_hat = Valid.copy()
        y_hat['naive']= dd[len(dd)- 1]
        for i in range(len(Valid)): 
            y_hat['naive'][i]= dd[len(dd) - len(Valid) + i]
        naive_rmse = rmse(Valid.footfall, y_hat.naive)


        # Moving Average Approach    
        moving_avg(4)
        avg4_rmse = rmse(Valid.footfall, y_hat.avg)
        moving_avg(6)
        avg6_rmse = rmse(Valid.footfall, y_hat.avg)    
        moving_avg(8)
        avg8_rmse = rmse(Valid.footfall, y_hat.avg)
        moving_avg(10)
        avg10_rmse = rmse(Valid.footfall, y_hat.avg)

        avg_rmse = min(avg4_rmse, avg6_rmse, avg8_rmse, avg10_rmse) 
        ma_temp = [avg4_rmse, avg6_rmse, avg8_rmse, avg10_rmse] 
        windows = (ma_temp.index(avg_rmse) + 2) * 2   
        moving_avg(windows)


        # Exponential Smoothing
        y_hat2 = valid.copy()
        fit1 = ExponentialSmoothing(np.asarray(train.footfall), seasonal_periods = 7, trend = 'add', seasonal= 'add').fit()
        y_hat2['Holt_Winter'] = fit1.forecast(len(valid))

        pred_d = y_hat2.Holt_Winter.to_frame()
        pred_d = pred_d.reset_index(level = ['timestamp'])
        pred_d['month'] = pred_d.timestamp.dt.month
        pred_d['day'] = pred_d.timestamp.dt.day
        temp1 = pd.merge(merge, pred_d, on = ('month','day'), how = 'left')
        temp1['prediction'] = temp1['ratio'] * temp1['Holt_Winter'] * 24
        temp1.index = temp1.timestamp_x
        y_hat['Holt_Winter'] = temp1['prediction']
        holt_winter_rmse = rmse(Valid.footfall, y_hat['Holt_Winter'])

        # SARIMA
        arima_model = auto_arima(train.footfall, X = None, start_p = 0, d = 1, 
                         start_q = 0, max_p = 5, max_d=2, max_q=5, start_P = 0, 
                         D = 1, start_Q = 0, max_P = 5, max_D = 5, max_Q = 5, 
                         m = 7, seasonal = True, stepwise = True, n_jobs = 1, 
                         error_action = 'warn', trace = True, random = False, 
                         random_state = 20, n_fits = 50 )
        
        y_hat2['SARIMA'] = arima_model.predict(n_periods = len(valid))
        
        pred_d = pd.DataFrame(y_hat2['SARIMA'], index = valid.index)
        pred_d = pred_d.reset_index(level = ['timestamp'])
        pred_d['month'] = pred_d.timestamp.dt.month
        pred_d['day'] = pred_d.timestamp.dt.day
        temp2 = pd.merge(merge, pred_d, on = ('month','day'), how = 'left')
        temp2['prediction'] = temp2['ratio'] * temp2['SARIMA'] * 24
        temp2.index = temp2.timestamp_x
        y_hat['SARIMA'] = temp2['prediction']
        sarima_rmse = rmse(Valid.footfall, y_hat['SARIMA'])

        # LSTM 
        train = temp.loc[(temp.timestamp >= '2018-01-01 00:00') & (temp.timestamp <= '2019-07-28 23:00')][[
            'footfall','month','day','hour','day_of_week']]
        test = temp.loc[(temp.timestamp >= '2019-07-28 14:00') & (temp.timestamp <= '2019-08-18 23:00')][[
            'footfall','month','day','hour','day_of_week']]  

        ff_transformer = RobustScaler()
        ff_transformer = ff_transformer.fit(train[['footfall']])

        train['footfall'] = ff_transformer.transform(train[['footfall']])
        test['footfall'] = ff_transformer.transform(test[['footfall']])

        time_steps = 10
        X_train, y_train = create_dataset(train, train.footfall, time_steps)
        X_test, y_test = create_dataset(test, test.footfall, time_steps)

        model = keras.Sequential()
        model.add(
          keras.layers.Bidirectional(
            keras.layers.LSTM(
              units=128, 
              input_shape=(X_train.shape[1], X_train.shape[2])
            )
          )
        )
        model.add(keras.layers.Dropout(rate=0.2))
        model.add(keras.layers.Dense(units=1))
        model.compile(loss='mean_squared_error', optimizer='adam')    

        history = model.fit(
        X_train, y_train, 
        epochs = 30, 
        batch_size = 32, 
        validation_split = 0.1,
        shuffle = False
        )

        y_pred = model.predict(X_test)

        y_train_inv = ff_transformer.inverse_transform(y_train.reshape(1, -1))
        y_test_inv = ff_transformer.inverse_transform(y_test.reshape(1, -1))
        y_pred_inv = ff_transformer.inverse_transform(y_pred)

        y_hat['LSTM'] =  y_pred_inv.flatten()
        lstm_rmse = rmse(y_test_inv.flatten(), y_pred_inv.flatten())

        # save prediction
        pred_loc = y_hat[['naive','avg','Holt_Winter','SARIMA','LSTM']]
        pred_loc.to_csv( 'pred/'+ str(loc) + '.csv')

        # DM-Test
        dm_NM = dm_test(Valid.footfall, y_hat['naive'], y_hat['avg'], h = 1, crit="MSE")
        dm_NE = dm_test(Valid.footfall, y_hat['naive'], y_hat['Holt_Winter'], h = 1, crit="MSE")    
        dm_NS = dm_test(Valid.footfall, y_hat['naive'], y_hat['SARIMA'], h = 1, crit="MSE")  
        dm_NL = dm_test(Valid.footfall, y_hat['naive'], y_pred_inv.flatten(), h = 1, crit="MSE")   
        dm_ME = dm_test(Valid.footfall, y_hat['avg'], y_hat['Holt_Winter'], h = 1, crit="MSE")   
        dm_MS = dm_test(Valid.footfall, y_hat['avg'], y_hat['SARIMA'], h = 1, crit="MSE")   
        dm_ML = dm_test(Valid.footfall, y_hat['avg'],  y_pred_inv.flatten(), h = 1, crit="MSE")   
        dm_ES = dm_test(Valid.footfall, y_hat['Holt_Winter'], y_hat['SARIMA'], h = 1, crit="MSE")    
        dm_EL = dm_test(Valid.footfall, y_hat['Holt_Winter'], y_pred_inv.flatten(), h = 1, crit="MSE")  
        dm_SL = dm_test(Valid.footfall, y_hat['SARIMA'], y_pred_inv.flatten(), h = 1, crit="MSE")

        dmtest = dmtest.append({'location': loc, 'nm': dm_NM.DM, 'p_nm': dm_NM.p_value, 
                                'ne': dm_NE.DM, 'p_ne': dm_NE.p_value, 'ns': dm_NS.DM, 'p_ns': dm_NS.p_value, 
                                'nl': dm_NL.DM, 'p_nl': dm_NL.p_value, 'me': dm_ME.DM, 'p_me': dm_ME.p_value, 
                                'ms': dm_MS.DM, 'p_ms': dm_MS.p_value, 'ml': dm_ML.DM, 'p_ml': dm_ML.p_value, 
                                'es': dm_ES.DM, 'p_es': dm_ES.p_value, 'el': dm_EL.DM, 'p_el': dm_EL.p_value, 
                                'sl': dm_SL.DM, 'p_sl': dm_SL.p_value}, ignore_index=True)


        pred_accuracy = pred_accuracy.append({'location': loc, 'Naive': naive_rmse, 'Moving_Average': avg_rmse, 
                                            'Expoential_Smoothing': holt_winter_rmse, 'SARIMA': sarima_rmse, 
                                            'LSTM': lstm_rmse}, ignore_index=True)


Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[7]             : AIC=5264.519, Time=0.34 sec
 ARIMA(1,1,0)(1,1,0)[7]             : AIC=5043.864, Time=0.14 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=4855.167, Time=0.24 sec
 ARIMA(0,1,1)(0,1,0)[7]             : AIC=5149.367, Time=0.06 sec
 ARIMA(0,1,1)(1,1,1)[7]             : AIC=4857.109, Time=0.38 sec
 ARIMA(0,1,1)(0,1,2)[7]             : AIC=4857.120, Time=0.60 sec
 ARIMA(0,1,1)(1,1,0)[7]             : AIC=4979.445, Time=0.12 sec
 ARIMA(0,1,1)(1,1,2)[7]             : AIC=4858.678, Time=1.10 sec
 ARIMA(0,1,0)(0,1,1)[7]             : AIC=4966.589, Time=0.11 sec
 ARIMA(1,1,1)(0,1,1)[7]             : AIC=4815.238, Time=0.45 sec
 ARIMA(1,1,1)(0,1,0)[7]             : AIC=inf, Time=0.32 sec
 ARIMA(1,1,1)(1,1,1)[7]             : AIC=4815.866, Time=0.79 sec
 ARIMA(1,1,1)(0,1,2)[7]             : AIC=4816.144, Time=0.99 sec
 ARIMA(1,1,1)(1,1,0)[7]             : AIC=inf, Time=0.75 sec
 ARIMA(1,1,1)(1,1,2)[7]             : AIC=i

 ARIMA(4,1,0)(4,1,1)[7]             : AIC=inf, Time=4.46 sec
 ARIMA(5,1,0)(5,1,0)[7]             : AIC=6444.091, Time=2.37 sec
 ARIMA(5,1,0)(4,1,0)[7]             : AIC=6454.422, Time=1.82 sec
 ARIMA(5,1,0)(5,1,1)[7]             : AIC=inf, Time=7.88 sec
 ARIMA(5,1,0)(4,1,1)[7]             : AIC=inf, Time=6.03 sec
 ARIMA(5,1,1)(5,1,0)[7]             : AIC=inf, Time=5.73 sec
 ARIMA(4,1,1)(5,1,0)[7]             : AIC=inf, Time=7.01 sec
 ARIMA(5,1,0)(5,1,0)[7] intercept   : AIC=6446.080, Time=6.31 sec

Best model:  ARIMA(5,1,0)(5,1,0)[7]          
Total fit time: 92.951 seconds
Epoch 1/30
388/388 [==============================] - 4s 10ms/step - loss: 0.5116 - val_loss: 0.5504
Epoch 2/30
388/388 [==============================] - 3s 9ms/step - loss: 0.2763 - val_loss: 0.3813
Epoch 3/30
388/388 [==============================] - 3s 9ms/step - loss: 0.2270 - val_loss: 0.3506
Epoch 4/30
388/388 [==============================] - 3s 9ms/step - loss: 0.2158 - val_loss: 0.3416
Epoch 5/30
388/388

385/385 [==============================] - 4s 10ms/step - loss: 0.1176 - val_loss: 0.1406
Epoch 18/30
385/385 [==============================] - 4s 10ms/step - loss: 0.1138 - val_loss: 0.1374
Epoch 19/30
385/385 [==============================] - 4s 10ms/step - loss: 0.1100 - val_loss: 0.1427
Epoch 20/30
385/385 [==============================] - 4s 10ms/step - loss: 0.1109 - val_loss: 0.1336
Epoch 21/30
385/385 [==============================] - 4s 10ms/step - loss: 0.1069 - val_loss: 0.1332
Epoch 22/30
385/385 [==============================] - 4s 10ms/step - loss: 0.1057 - val_loss: 0.1286
Epoch 23/30
385/385 [==============================] - 4s 10ms/step - loss: 0.1019 - val_loss: 0.1226
Epoch 24/30
385/385 [==============================] - 4s 10ms/step - loss: 0.0995 - val_loss: 0.1124
Epoch 25/30
385/385 [==============================] - 4s 10ms/step - loss: 0.0976 - val_loss: 0.1141
Epoch 26/30
385/385 [==============================] - 4s 10ms/step - loss: 0.0962 - val_loss:

 ARIMA(5,1,0)(4,1,1)[7]             : AIC=inf, Time=5.07 sec
 ARIMA(5,1,1)(5,1,0)[7]             : AIC=inf, Time=13.76 sec
 ARIMA(4,1,1)(5,1,0)[7]             : AIC=inf, Time=13.59 sec
 ARIMA(5,1,0)(5,1,0)[7] intercept   : AIC=5356.250, Time=5.50 sec

Best model:  ARIMA(5,1,0)(5,1,0)[7]          
Total fit time: 131.140 seconds
Epoch 1/30
388/388 [==============================] - 5s 14ms/step - loss: 0.2475 - val_loss: 0.1399
Epoch 2/30
388/388 [==============================] - 5s 13ms/step - loss: 0.1148 - val_loss: 0.1201
Epoch 3/30
388/388 [==============================] - 5s 13ms/step - loss: 0.0903 - val_loss: 0.1107
Epoch 4/30
388/388 [==============================] - 5s 13ms/step - loss: 0.0749 - val_loss: 0.1067
Epoch 5/30
388/388 [==============================] - 5s 13ms/step - loss: 0.0716 - val_loss: 0.1103
Epoch 6/30
388/388 [==============================] - 5s 13ms/step - loss: 0.0691 - val_loss: 0.1070
Epoch 7/30
388/388 [==============================] - 5s 13ms/st

387/387 [==============================] - 6s 16ms/step - loss: 0.0585 - val_loss: 0.0516
Epoch 16/30
387/387 [==============================] - 6s 15ms/step - loss: 0.0557 - val_loss: 0.0510
Epoch 17/30
387/387 [==============================] - 6s 15ms/step - loss: 0.0558 - val_loss: 0.0519
Epoch 18/30
387/387 [==============================] - 6s 15ms/step - loss: 0.0563 - val_loss: 0.0514
Epoch 19/30
387/387 [==============================] - 6s 15ms/step - loss: 0.0555 - val_loss: 0.0512
Epoch 20/30
387/387 [==============================] - 6s 15ms/step - loss: 0.0526 - val_loss: 0.0506
Epoch 21/30
387/387 [==============================] - 6s 15ms/step - loss: 0.0538 - val_loss: 0.0535
Epoch 22/30
387/387 [==============================] - 6s 15ms/step - loss: 0.0528 - val_loss: 0.0521
Epoch 23/30
387/387 [==============================] - 6s 15ms/step - loss: 0.0507 - val_loss: 0.0511
Epoch 24/30
387/387 [==============================] - 6s 15ms/step - loss: 0.0485 - val_loss:

...

 ARIMA(0,1,0)(4,1,0)[7]             : AIC=3618.051, Time=0.48 sec
 ARIMA(2,1,0)(4,1,0)[7]             : AIC=3531.392, Time=0.72 sec
 ARIMA(2,1,0)(3,1,0)[7]             : AIC=3546.569, Time=0.50 sec
 ARIMA(2,1,0)(5,1,0)[7]             : AIC=3532.948, Time=1.22 sec
 ARIMA(2,1,0)(4,1,1)[7]             : AIC=inf, Time=4.18 sec
 ARIMA(2,1,0)(3,1,1)[7]             : AIC=inf, Time=1.32 sec
 ARIMA(2,1,0)(5,1,1)[7]             : AIC=inf, Time=7.10 sec
 ARIMA(3,1,0)(4,1,0)[7]             : AIC=3528.086, Time=1.00 sec
 ARIMA(3,1,0)(3,1,0)[7]             : AIC=3541.840, Time=0.51 sec
 ARIMA(3,1,0)(5,1,0)[7]             : AIC=3529.286, Time=1.36 sec
 ARIMA(3,1,0)(4,1,1)[7]             : AIC=inf, Time=5.56 sec
 ARIMA(3,1,0)(3,1,1)[7]             : AIC=inf, Time=1.41 sec
 ARIMA(3,1,0)(5,1,1)[7]             : AIC=inf, Time=7.96 sec
 ARIMA(4,1,0)(4,1,0)[7]             : AIC=3510.154, Time=1.07 sec
 ARIMA(4,1,0)(3,1,0)[7]             : AIC=3527.157, Time=0.72 sec
 ARIMA(4,1,0)(5,1,0)[7]             : AI

249/249 [==============================] - 5s 21ms/step - loss: 0.0367 - val_loss: 0.0473
Epoch 11/30
249/249 [==============================] - 5s 21ms/step - loss: 0.0365 - val_loss: 0.0387
Epoch 12/30
249/249 [==============================] - 5s 21ms/step - loss: 0.0349 - val_loss: 0.0385
Epoch 13/30
249/249 [==============================] - 5s 21ms/step - loss: 0.0341 - val_loss: 0.0389
Epoch 14/30
249/249 [==============================] - 5s 21ms/step - loss: 0.0332 - val_loss: 0.0397
Epoch 15/30
249/249 [==============================] - 5s 21ms/step - loss: 0.0335 - val_loss: 0.0355
Epoch 16/30
249/249 [==============================] - 5s 21ms/step - loss: 0.0328 - val_loss: 0.0364
Epoch 17/30
249/249 [==============================] - 5s 21ms/step - loss: 0.0335 - val_loss: 0.0358
Epoch 18/30
249/249 [==============================] - 5s 21ms/step - loss: 0.0325 - val_loss: 0.0366
Epoch 19/30
249/249 [==============================] - 5s 21ms/step - loss: 0.0328 - val_loss:

 ARIMA(1,1,0)(2,1,0)[7]             : AIC=3319.519, Time=0.22 sec
 ARIMA(1,1,0)(3,1,0)[7]             : AIC=3307.648, Time=0.35 sec
 ARIMA(1,1,0)(4,1,0)[7]             : AIC=3285.063, Time=0.80 sec
 ARIMA(1,1,0)(5,1,0)[7]             : AIC=3286.653, Time=1.05 sec
 ARIMA(1,1,0)(4,1,1)[7]             : AIC=inf, Time=2.80 sec
 ARIMA(1,1,0)(3,1,1)[7]             : AIC=inf, Time=0.94 sec
 ARIMA(1,1,0)(5,1,1)[7]             : AIC=inf, Time=5.17 sec
 ARIMA(0,1,0)(4,1,0)[7]             : AIC=3352.889, Time=0.41 sec
 ARIMA(2,1,0)(4,1,0)[7]             : AIC=3262.329, Time=0.71 sec
 ARIMA(2,1,0)(3,1,0)[7]             : AIC=3279.487, Time=0.50 sec
 ARIMA(2,1,0)(5,1,0)[7]             : AIC=3263.704, Time=1.30 sec
 ARIMA(2,1,0)(4,1,1)[7]             : AIC=inf, Time=2.77 sec
 ARIMA(2,1,0)(3,1,1)[7]             : AIC=inf, Time=1.91 sec
 ARIMA(2,1,0)(5,1,1)[7]             : AIC=inf, Time=6.05 sec
 ARIMA(3,1,0)(4,1,0)[7]             : AIC=3250.127, Time=0.89 sec
 ARIMA(3,1,0)(3,1,0)[7]             : AI

 ARIMA(4,1,0)(5,1,0)[7]             : AIC=2815.504, Time=1.30 sec
 ARIMA(4,1,0)(4,1,0)[7]             : AIC=2820.637, Time=0.83 sec
 ARIMA(4,1,0)(5,1,1)[7]             : AIC=inf, Time=8.74 sec
 ARIMA(4,1,0)(4,1,1)[7]             : AIC=inf, Time=4.96 sec
 ARIMA(5,1,0)(5,1,0)[7]             : AIC=2811.589, Time=1.43 sec
 ARIMA(5,1,0)(4,1,0)[7]             : AIC=2815.986, Time=1.09 sec
 ARIMA(5,1,0)(5,1,1)[7]             : AIC=inf, Time=8.39 sec
 ARIMA(5,1,0)(4,1,1)[7]             : AIC=inf, Time=6.68 sec
 ARIMA(5,1,1)(5,1,0)[7]             : AIC=2804.837, Time=5.07 sec
 ARIMA(5,1,1)(4,1,0)[7]             : AIC=2808.658, Time=4.35 sec
 ARIMA(5,1,1)(5,1,1)[7]             : AIC=inf, Time=9.86 sec
 ARIMA(5,1,1)(4,1,1)[7]             : AIC=inf, Time=7.22 sec
 ARIMA(4,1,1)(5,1,0)[7]             : AIC=2805.357, Time=3.21 sec
 ARIMA(5,1,2)(5,1,0)[7]             : AIC=2806.430, Time=7.21 sec
 ARIMA(4,1,2)(5,1,0)[7]             : AIC=2807.206, Time=5.77 sec
 ARIMA(5,1,1)(5,1,0)[7] intercept   : AI

239/239 [==============================] - 7s 30ms/step - loss: 0.1419 - val_loss: 0.0337
Epoch 3/30
239/239 [==============================] - 7s 29ms/step - loss: 0.0787 - val_loss: 0.0342
Epoch 4/30
239/239 [==============================] - 7s 29ms/step - loss: 0.0607 - val_loss: 0.0398
Epoch 5/30
239/239 [==============================] - 7s 29ms/step - loss: 0.0479 - val_loss: 0.0152
Epoch 6/30
239/239 [==============================] - 7s 29ms/step - loss: 0.0393 - val_loss: 0.0154
Epoch 7/30
239/239 [==============================] - 7s 29ms/step - loss: 0.0364 - val_loss: 0.0127
Epoch 8/30
239/239 [==============================] - 7s 29ms/step - loss: 0.0325 - val_loss: 0.0134
Epoch 9/30
239/239 [==============================] - 7s 29ms/step - loss: 0.0304 - val_loss: 0.0117
Epoch 10/30
239/239 [==============================] - 7s 29ms/step - loss: 0.0260 - val_loss: 0.0114
Epoch 11/30
239/239 [==============================] - 7s 30ms/step - loss: 0.0250 - val_loss: 0.0118

In [13]:
pred_accuracy = pred_accuracy.drop_duplicates(subset='location')
pred_accuracy

,location,Naive,Moving_Average,Expoential_Smoothing,SARIMA,LSTM
0,1021.0,33.840224,30.319429,28.353869,28.806737,26.202414
1,1024.0,249.732868,144.770955,190.207385,199.335860,111.382302
2,1026.0,100.960408,89.324404,70.610780,103.079917,88.967293
3,1030.0,61.342089,40.292335,37.028043,39.952837,40.858323
4,1038.0,54.176620,45.053754,53.154440,48.977382,45.790956
5,1042.0,66.003677,53.313289,52.465733,55.178499,50.576620
6,1043.0,101.408079,78.677691,71.203320,72.666655,70.202263
7,1044.0,21.953827,15.794267,16.419474,21.568020,16.247101
8,1046.0,48.350982,35.907702,36.816764,37.687269,33.852473
9,1050.0,191.608937,193.534345,181.445642,181.550143,134.071649


In [14]:
dmtest = dmtest.drop_duplicates(subset='location')
dmtest

,location,nm,p_nm,ne,p_ne,ns,p_ns,nl,p_nl,me,...,ms,p_ms,ml,p_ml,es,p_es,el,p_el,sl,p_sl
0,1021.0,1.982799,4.793348e-02,3.056086,2.361614e-03,2.797316,5.350380e-03,3.578053,3.796634e-04,2.174526,...,1.634331,1.028151e-01,2.781715,5.610286e-03,-4.208823,3.040782e-05,1.532835,1.259452e-01,1.765811,7.803412e-02
1,1024.0,3.453049,6.010470e-04,2.309771,2.130485e-02,2.011105,4.484737e-02,3.871005,1.226706e-04,-5.216060,...,-5.935460,5.463109e-09,3.922601,9.976591e-05,-5.137299,3.993809e-07,7.422161,4.951809e-13,8.041510,6.400433e-15
2,1026.0,2.249372,2.492057e-02,6.020304,3.359941e-09,-0.443629,6.575016e-01,2.198543,2.836509e-02,4.789887,...,-2.689290,7.398194e-03,0.133680,8.937092e-01,-11.809221,1.415921e-28,-4.526992,7.476473e-06,2.731662,6.523052e-03
3,1030.0,7.954802,1.195232e-14,7.179943,2.521834e-12,6.900186,1.568184e-11,5.524283,5.314740e-08,2.373328,...,0.267028,7.895571e-01,-0.244583,8.068788e-01,-2.865957,4.331643e-03,-2.261922,2.412823e-02,-0.424935,6.710661e-01
4,1038.0,2.137403,3.304673e-02,0.225213,8.219047e-01,1.186551,2.359650e-01,1.846160,6.545629e-02,-5.479530,...,-3.765017,1.862030e-04,-0.572749,5.670709e-01,5.473432,6.975767e-08,5.374234,1.178652e-07,3.022625,2.633710e-03
5,1042.0,3.963762,8.446217e-05,3.765666,1.857336e-04,3.297576,1.044221e-03,3.201096,1.455614e-03,0.517839,...,-1.177647,2.394942e-01,0.895107,3.711581e-01,-4.460505,1.009499e-05,0.759676,4.478039e-01,1.698450,9.004134e-02
6,1043.0,2.024891,4.340632e-02,2.555726,1.088957e-02,2.461237,1.418056e-02,2.675606,7.702606e-03,2.332089,...,1.974834,4.883339e-02,2.245546,2.516658e-02,-1.993409,4.675630e-02,0.338181,7.353677e-01,0.806887,4.201128e-01
7,1044.0,5.525869,5.269665e-08,5.011741,7.484025e-07,0.298822,7.651995e-01,4.934511,1.094183e-06,-1.007932,...,-4.570808,6.121633e-06,-0.820848,4.121216e-01,-6.098013,2.142054e-09,0.224921,8.221323e-01,4.011350,6.954371e-05
8,1046.0,5.441062,8.285252e-08,5.246786,2.285342e-07,4.920180,1.173448e-06,5.282642,1.899459e-07,-0.515188,...,-0.953423,3.408337e-01,0.949065,3.430433e-01,-1.922822,5.506677e-02,1.458755,1.452567e-01,1.803511,7.190624e-02
9,1050.0,-0.319042,7.498270e-01,1.440646,1.503069e-01,1.408207,1.596874e-01,6.611367,9.740086e-11,3.429515,...,3.207141,1.425973e-03,8.284617,1.081755e-15,-0.169569,8.654170e-01,6.984644,9.086461e-12,6.989470,8.806114e-12


In [15]:
dmtest.to_csv('dmtest_final2.csv')
pred_accuracy.to_csv('pred_accuracy_final2.csv')